In [1]:
from bs4 import BeautifulSoup
from datetime import date
import os
import pandas as pd
import requests
import re
import time
import traceback
    
# Returns a dataframe with urls for any odd lot forms filed for the given CIK_num
def get_odd_lot_form_url(CIK_num):
    try:
        # Search for the company with CIK number CIK_num
        search_results_url = "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=" + \
            CIK_num + "&type=SC+TO-I&dateb=&owner=exclude&count=100"
        result_site = requests.get(search_results_url)
        data = result_site.text
        EDGAR_results_page = BeautifulSoup(data, 'html.parser')

        # Get list of tags with the string \"SC TO-I\", this signals a tender offer document.\n",
        sc_to_i_tag_list = EDGAR_results_page.find_all("td", string="SC TO-I")
        # If no tender offer tags found, quit now
        if len(sc_to_i_tag_list) < 1:
            return None

        final_dates = []
        final_urls = []
        for tag in sc_to_i_tag_list:
            # Get the url to the details page
            filing_details_url = tag.find_next("a")['href']  # extension for the document
            detail_url = "https://www.sec.gov/" + filing_details_url
            print(detail_url)

            # Get the html of the details page
            details_site = requests.get(detail_url)
            details_site_data = details_site.text
            FILING_detail_page = BeautifulSoup(details_site_data, 'html.parser')

            # Get the tender offer/tender ammendment 
            tender_document_tag = FILING_detail_page.find('td', string="Complete submission text file")
            tender_document_url = "https://www.sec.gov/" + tender_document_tag.find_next("a")['href']
            form = requests.get(tender_document_url)

            # If this tender offer has no odd lot provision, then skip this iteration and do
            # not add it
            if re.search(r'(odd lot)', form.text, re.IGNORECASE) is None:
                continue

            # Get date of tender offer
            row_cells = tag.find_next_siblings("td")
            date = pd.to_datetime(row_cells[2].get_text(), format='%Y-%m-%d', errors='ignore').date()

            # Add the information
            final_dates.append(date)
            final_urls.append(detail_url)
            
        if len(final_urls) < 1:
            return None
        # Return the information as a dataframe
        return pd.DataFrame({'cik': '"' + str(CIK_num) + '"', 'date': final_dates, 'url': final_urls})
    except:
        print("Exception occured while attempting to produce submission text file, returning None")
        traceback.print_exc()
        return None

# Returns a dataframe with general information about tender offers
def get_tender_offer_data(CIK_nums):
    ciks = []
    dates = []
    urls = []
    
    save_point = int(len(CIK_nums) / 10) # Set the number of iterations before saving
    save_count = 0 # Keeps track of current iteration, resets when 
    total_count = 0
    print("save point: " + str(save_point))
    
    for CIK_num in CIK_nums:
        try:
            # Save data if past the save point
            total_count += 1
            save_count += 1
            if save_count >save_point:
                save_count = 0
                tender_offer_data = pd.DataFrame({'cik': ciks, 'date': dates, 'url': urls})
                tender_offer_data.to_csv("temp.csv", index=False)
                print("\n\n\n\n\n\nSaved data at total count: " + str(total_count) + "; rows: " + str(len(urls)) + "\n\n\n\n\n\n")
                
            # Search for the company with CIK number CIK_num
            search_results_url = "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=" + \
                CIK_num + "&type=SC+TO-I&dateb=&owner=exclude&count=100"
            result_site = requests.get(search_results_url)
            data = result_site.text
            EDGAR_results_page = BeautifulSoup(data, 'html.parser')

            # Get list of tags with the string \"SC TO-I\", this signals a tender offer document.\n",
            sc_to_i_tag_list = EDGAR_results_page.find_all("td", string="SC TO-I")
            # If no tender offer tags found, quit now
            if len(sc_to_i_tag_list) < 1:
                continue

            for tag in sc_to_i_tag_list:
                # Get the url to the details page
                filing_details_url = tag.find_next("a")['href']  # extension for the document
                detail_url = "https://www.sec.gov/" + filing_details_url
                print(detail_url)

#                 # Get the html of the details page
#                 details_site = requests.get(detail_url)
#                 details_site_data = details_site.text
#                 FILING_detail_page = BeautifulSoup(details_site_data, 'html.parser')
# 
#                 # Get the tender offer/tender ammendment 
#                 tender_document_tag = FILING_detail_page.find('td', string="Complete submission text file")
#                 tender_document_url = "https://www.sec.gov/" + tender_document_tag.find_next("a")['href']
#                 form = requests.get(tender_document_url)
# 
#                 # If this tender offer has no odd lot provision, then skip this iteration and do
#                 # not add it
#                 if re.search(r'(odd lot)', form.text, re.IGNORECASE) is None:
#                     continue

                # Get date of tender offer
                row_cells = tag.find_next_siblings("td")
                date = pd.to_datetime(row_cells[2].get_text(), format='%Y-%m-%d', errors='ignore').date()

                # Add the information
                ciks.append('"' + str(CIK_num) + '""')
                dates.append(date)
                urls.append(detail_url)   
                
        except:
            print("Exception occured while attempting to produce submission text file, returning None")
            traceback.print_exc()
    
    return pd.DataFrame({'cik': ciks, 'date': dates, 'url': urls})

# csv1 should be the path to the first csv, csv2 should be the path to the second csv,
# and out should be the path the concatenated files are written to
def concat_csvs(csv1, csv2, out):
    csv1_df = pd.read_csv(csv1)
    csv2_df = pd.read_csv(csv2)
    pd.concat([csv1_df, csv2_df]).to_csv(out, index=False)

def main():
    print("Beginning running")
    with open("CIK_Numbers.txt", "r") as cik_numbers_file: 
        cik_numbers = cik_numbers_file.readlines()
        print("CIK count: " + str(len(cik_numbers)))
        
        # May need to go through 100000 to 200000 again
        cik_numbers = cik_numbers[600000:]
        try:
            tender_offer_data = get_tender_offer_data(cik_numbers)
            tender_offer_data.to_csv("tender_offer_data_7.csv", index=False)
            """ 
            ### Get urls for odd lot forms
            url_data = pd.DataFrame(columns={'cik': [], 'date': [], 'url': []})
            try:
                for cik_number in cik_numbers:
                    df = get_odd_lot_form_urls(cik_number.rstrip())
                    if df is not None and not df.empty:
                        url_data = url_data.append(df, ignore_index=True)
                print(url_data)
                url_data.to_csv("url_data_8
                
                
                .csv", index=False)
            except:
                # If there is an exception, save what we have
                print("Exception occured. Writing now to not lose data")
                url_data.to_csv("url_data_8.csv", index=False)
                traceback.print_exc()
            """
        except:
            print("get_tender_offer_data failed")
            traceback.print_exc()

            
if __name__ == "__main__": 
    main()

Beginning running
CIK count: 708280
save point: 10828
https://www.sec.gov//Archives/edgar/data/1016470/000104746908009760/0001047469-08-009760-index.htm
https://www.sec.gov//Archives/edgar/data/745308/000119312515299386/0001193125-15-299386-index.htm
https://www.sec.gov//Archives/edgar/data/745308/000119312515299386/0001193125-15-299386-index.htm
https://www.sec.gov//Archives/edgar/data/745308/000119312515299386/0001193125-15-299386-index.htm
https://www.sec.gov//Archives/edgar/data/893538/000104746912002086/0001047469-12-002086-index.htm
https://www.sec.gov//Archives/edgar/data/86312/000104746904002840/0001047469-04-002840-index.htm
https://www.sec.gov//Archives/edgar/data/86312/000104746904002840/0001047469-04-002840-index.htm
https://www.sec.gov//Archives/edgar/data/316747/000104746904002862/0001047469-04-002862-index.htm
https://www.sec.gov//Archives/edgar/data/316747/000104746904002840/0001047469-04-002840-index.htm
https://www.sec.gov//Archives/edgar/data/86312/000104746904002840

Traceback (most recent call last):
  File "C:\Users\adachille\Anaconda3\lib\site-packages\urllib3\contrib\pyopenssl.py", line 444, in wrap_socket
    cnx.do_handshake()
  File "C:\Users\adachille\Anaconda3\lib\site-packages\OpenSSL\SSL.py", line 1907, in do_handshake
    self._raise_ssl_error(self._ssl, result)
  File "C:\Users\adachille\Anaconda3\lib\site-packages\OpenSSL\SSL.py", line 1631, in _raise_ssl_error
    raise SysCallError(errno, errorcode.get(errno))
OpenSSL.SSL.SysCallError: (10053, 'WSAECONNABORTED')

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\adachille\Anaconda3\lib\site-packages\urllib3\connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "C:\Users\adachille\Anaconda3\lib\site-packages\urllib3\connectionpool.py", line 343, in _make_request
    self._validate_conn(conn)
  File "C:\Users\adachille\Anaconda3\lib\site-packages\urllib3\connectionpool.py", line 849, in _validate_con

https://www.sec.gov//Archives/edgar/data/1289340/000119312509176378/0001193125-09-176378-index.htm
https://www.sec.gov//Archives/edgar/data/913241/000101905608000298/0001019056-08-000298-index.htm
https://www.sec.gov//Archives/edgar/data/878522/000119312513463708/0001193125-13-463708-index.htm
https://www.sec.gov//Archives/edgar/data/720672/000095013403012488/0000950134-03-012488-index.htm
https://www.sec.gov//Archives/edgar/data/931948/000119312517173162/0001193125-17-173162-index.htm
https://www.sec.gov//Archives/edgar/data/931948/000134100413000222/0001341004-13-000222-index.htm
https://www.sec.gov//Archives/edgar/data/931948/000095013303002464/0000950133-03-002464-index.htm
https://www.sec.gov//Archives/edgar/data/928953/000095015203005168/0000950152-03-005168-index.htm
https://www.sec.gov//Archives/edgar/data/1473597/000104746914000485/0001047469-14-000485-index.htm
https://www.sec.gov//Archives/edgar/data/1473597/000104746914000485/0001047469-14-000485-index.htm
https://www.sec.g

https://www.sec.gov//Archives/edgar/data/1361709/000119312510129291/0001193125-10-129291-index.htm
https://www.sec.gov//Archives/edgar/data/1397016/000114420412040126/0001144204-12-040126-index.htm
https://www.sec.gov//Archives/edgar/data/1527590/000114420416125570/0001144204-16-125570-index.htm
https://www.sec.gov//Archives/edgar/data/813623/000119312518304312/0001193125-18-304312-index.htm
https://www.sec.gov//Archives/edgar/data/813623/000139834417004033/0001398344-17-004033-index.htm
https://www.sec.gov//Archives/edgar/data/813623/000119312514008956/0001193125-14-008956-index.htm
https://www.sec.gov//Archives/edgar/data/813623/000119312511155808/0001193125-11-155808-index.htm
https://www.sec.gov//Archives/edgar/data/813623/000119312518304312/0001193125-18-304312-index.htm
https://www.sec.gov//Archives/edgar/data/813623/000139834417004033/0001398344-17-004033-index.htm
https://www.sec.gov//Archives/edgar/data/813623/000119312514008956/0001193125-14-008956-index.htm
https://www.sec.g

https://www.sec.gov//Archives/edgar/data/1167257/000111667904002145/0001116679-04-002145-index.htm
https://www.sec.gov//Archives/edgar/data/1603480/000119312518169720/0001193125-18-169720-index.htm
https://www.sec.gov//Archives/edgar/data/1173431/000095013707002638/0000950137-07-002638-index.htm
https://www.sec.gov//Archives/edgar/data/1055455/000119312505130034/0001193125-05-130034-index.htm
https://www.sec.gov//Archives/edgar/data/860235/000095015200008261/0000950152-00-008261-index.htm
https://www.sec.gov//Archives/edgar/data/860235/000095015200004743/0000950152-00-004743-index.htm
https://www.sec.gov//Archives/edgar/data/860235/000095015200008261/0000950152-00-008261-index.htm
https://www.sec.gov//Archives/edgar/data/860235/000095015200004743/0000950152-00-004743-index.htm
https://www.sec.gov//Archives/edgar/data/318833/000095012901500467/0000950129-01-500467-index.htm
https://www.sec.gov//Archives/edgar/data/860235/000095015200008261/0000950152-00-008261-index.htm
https://www.sec.

...

https://www.sec.gov//Archives/edgar/data/1279014/000116923206001639/0001169232-06-001639-index.htm
https://www.sec.gov//Archives/edgar/data/1279014/000116923205005684/0001169232-05-005684-index.htm
https://www.sec.gov//Archives/edgar/data/1279014/000116923205005684/0001169232-05-005684-index.htm
https://www.sec.gov//Archives/edgar/data/1279014/000116923205004688/0001169232-05-004688-index.htm
https://www.sec.gov//Archives/edgar/data/1279014/000116923205003224/0001169232-05-003224-index.htm
https://www.sec.gov//Archives/edgar/data/1279014/000116923205001769/0001169232-05-001769-index.htm
https://www.sec.gov//Archives/edgar/data/1279014/000090873704000985/0000908737-04-000985-index.htm
https://www.sec.gov//Archives/edgar/data/934739/000094627504000887/0000946275-04-000887-index.htm
https://www.sec.gov//Archives/edgar/data/1252849/000119312513395960/0001193125-13-395960-index.htm
https://www.sec.gov//Archives/edgar/data/766704/000119312514388435/0001193125-14-388435-index.htm
https://www.

https://www.sec.gov//Archives/edgar/data/1527541/000119312515223361/0001193125-15-223361-index.htm
https://www.sec.gov//Archives/edgar/data/106618/000092189514001694/0000921895-14-001694-index.htm
https://www.sec.gov//Archives/edgar/data/106640/000119312518134842/0001193125-18-134842-index.htm
https://www.sec.gov//Archives/edgar/data/1089876/000090266413003294/0000902664-13-003294-index.htm
https://www.sec.gov//Archives/edgar/data/1089876/000090266413001780/0000902664-13-001780-index.htm
https://www.sec.gov//Archives/edgar/data/1089876/000090266412001282/0000902664-12-001282-index.htm
https://www.sec.gov//Archives/edgar/data/1089876/000090266412000661/0000902664-12-000661-index.htm
https://www.sec.gov//Archives/edgar/data/1089876/000090266411001417/0000902664-11-001417-index.htm
https://www.sec.gov//Archives/edgar/data/1089876/000090266411000703/0000902664-11-000703-index.htm
https://www.sec.gov//Archives/edgar/data/1089876/000090266410003357/0000902664-10-003357-index.htm
https://www.

https://www.sec.gov//Archives/edgar/data/821407/000095014401506387/0000950144-01-506387-index.htm
https://www.sec.gov//Archives/edgar/data/949240/000095014406008664/0000950144-06-008664-index.htm
https://www.sec.gov//Archives/edgar/data/949240/000095014403008805/0000950144-03-008805-index.htm
https://www.sec.gov//Archives/edgar/data/949240/000095014406008664/0000950144-06-008664-index.htm
https://www.sec.gov//Archives/edgar/data/949240/000095014403008805/0000950144-03-008805-index.htm
https://www.sec.gov//Archives/edgar/data/1024520/000104746905017726/0001047469-05-017726-index.htm
https://www.sec.gov//Archives/edgar/data/1024520/000104746905017724/0001047469-05-017724-index.htm
https://www.sec.gov//Archives/edgar/data/1091907/000095012803001286/0000950128-03-001286-index.htm
https://www.sec.gov//Archives/edgar/data/1091907/000095012803001286/0000950128-03-001286-index.htm
https://www.sec.gov//Archives/edgar/data/723527/000100547702000180/0001005477-02-000180-index.htm
https://www.sec.

https://www.sec.gov//Archives/edgar/data/742550/000110465909055047/0001104659-09-055047-index.htm
https://www.sec.gov//Archives/edgar/data/742550/000110465909055047/0001104659-09-055047-index.htm
https://www.sec.gov//Archives/edgar/data/875159/000095016209000118/0000950162-09-000118-index.htm
https://www.sec.gov//Archives/edgar/data/875159/000095016206001146/0000950162-06-001146-index.htm
https://www.sec.gov//Archives/edgar/data/875159/000095016206000660/0000950162-06-000660-index.htm
https://www.sec.gov//Archives/edgar/data/1523700/000157104917005800/0001571049-17-005800-index.htm
https://www.sec.gov//Archives/edgar/data/875159/000095016209000118/0000950162-09-000118-index.htm
https://www.sec.gov//Archives/edgar/data/875159/000095016206001146/0000950162-06-001146-index.htm
https://www.sec.gov//Archives/edgar/data/875159/000095016206000660/0000950162-06-000660-index.htm
https://www.sec.gov//Archives/edgar/data/875159/000095016209000118/0000950162-09-000118-index.htm
https://www.sec.gov

https://www.sec.gov//Archives/edgar/data/1587221/000119312515378309/0001193125-15-378309-index.htm
https://www.sec.gov//Archives/edgar/data/1318008/000110465909039265/0001104659-09-039265-index.htm
https://www.sec.gov//Archives/edgar/data/812090/000157104917005407/0001571049-17-005407-index.htm
https://www.sec.gov//Archives/edgar/data/812090/000157104916020155/0001571049-16-020155-index.htm
https://www.sec.gov//Archives/edgar/data/812090/000157104916014463/0001571049-16-014463-index.htm
https://www.sec.gov//Archives/edgar/data/836412/000157104917005409/0001571049-17-005409-index.htm
https://www.sec.gov//Archives/edgar/data/836412/000157104916020157/0001571049-16-020157-index.htm
https://www.sec.gov//Archives/edgar/data/836412/000157104916014465/0001571049-16-014465-index.htm
https://www.sec.gov//Archives/edgar/data/1129425/000119312509235941/0001193125-09-235941-index.htm
